In [29]:
import pandas as pd


df_1 = pd.read_csv("UNSWNB15Full.csv")
df_2 = pd.read_csv("CICIDS_full/CICIDS2017_standardised.csv")

In [102]:
from sklearn.preprocessing import LabelEncoder
x = df_2.iloc[:, :-1]
y = df_2.iloc[:, -1]
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
class_names = label_encoder.classes_
print(f"✅ Labels encoded. Found {len(class_names)} classes.")

✅ Labels encoded. Found 8 classes.


In [31]:
df_1['ACK Flag Count']

0         28
1         10
2         11
3          7
4          7
          ..
447910     3
447911     0
447912     3
447913     3
447914     3
Name: ACK Flag Count, Length: 447915, dtype: int64

In [41]:
df_2 = pd.concat([x, df_y], axis=1)

df_2.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,38308,1,1,6,6,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,389,479,11,5,172,326,79,0,15.636364,31.449238,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
2,88,1095,10,6,3150,3150,1575,0,315.000000,632.561635,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
3,389,15206,17,12,3452,6660,1313,0,203.058823,425.778474,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
4,88,1092,9,6,3150,3152,1575,0,350.000000,694.509719,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0


In [22]:
def normalize_name(name):
    """
    Helper function to normalize a column name for robust comparison.
    Handles capitalization, whitespace, underscores, common abbreviations,
    and numerical suffixes.
    """
    # Define a dictionary of replacements for common abbreviations and typos
    replacements = {
        'forward': 'fwd',
        'backward': 'bwd',
        'packets': 'pkt',
        'packet': 'pkt',
        'pkts': 'pkt',
        'average': 'avg',
        'min': 'min',
        'max': 'max',
        'std': 'std',
        'win': 'win',
        'cwr': 'cwe'
    }

    # Convert to lowercase and handle underscores and spaces
    normalized_name = name.lower().strip().replace('_', ' ')
    
    # Replace full words with abbreviations
    for old, new in replacements.items():
        normalized_name = normalized_name.replace(old, new)
        
    # Remove trailing numerical suffixes like '.1' or '.2'
    normalized_name = re.sub(r'\.\d+$', '', normalized_name)
    
    # Split into words, sort, and join
    words = normalized_name.split()
    return ''.join(sorted(words))

In [23]:
import re

def compare_df_columns(df1: pd.DataFrame, df2: pd.DataFrame) -> dict:
    """
    Compares the column names of two pandas DataFrames and provides detailed
    information about any mismatches. This function normalizes names by
    ignoring leading/trailing whitespace, capitalization, and specific
    abbreviations ('forward'/'fwd', 'backward'/'bwd', 'packets'/'pkt', etc.)
    and trailing numerical suffixes (e.g., '.1').

    Args:
        df1 (pd.DataFrame): The first DataFrame.
        df2 (pd.DataFrame): The second DataFrame.

    Returns:
        dict: A dictionary containing the following keys:
              - 'are_identical': True if columns are identical (name and order), False otherwise.
              - 'in_df1_only': A list of original column names present only in df1.
              - 'in_df2_only': A list of original column names present only in df2.
              - 'order_mismatch': True if normalized column names are the same but the order differs.
    """
    def normalize_name(name):
        """
        Helper function to normalize a column name for robust comparison.
        Handles capitalization, whitespace, underscores, common abbreviations,
        and numerical suffixes.
        """
        # Define a dictionary of replacements for common abbreviations and typos
        replacements = {
            'forward': 'fwd',
            'backward': 'bwd',
            'packets': 'pkt',
            'packet': 'pkt',
            'pkts': 'pkt',
            'average': 'avg',
            'min': 'min',
            'max': 'max',
            'std': 'std',
            'win': 'win',
            'cwr': 'cwe'
        }

        # Convert to lowercase and handle underscores and spaces
        normalized_name = name.lower().strip().replace('_', ' ')
        
        # Replace full words with abbreviations
        for old, new in replacements.items():
            normalized_name = normalized_name.replace(old, new)
            
        # Remove trailing numerical suffixes like '.1' or '.2'
        normalized_name = re.sub(r'\.\d+$', '', normalized_name)
        
        # Split into words, sort, and join
        words = normalized_name.split()
        return ''.join(sorted(words))

    # Normalize the column names for a more flexible comparison
    df1_normalized = [normalize_name(col) for col in df1.columns.tolist()]
    df2_normalized = [normalize_name(col) for col in df2.columns.tolist()]
    
    # Get original column names for the output
    df1_cols = df1.columns.tolist()
    df2_cols = df2.columns.tolist()

    # Check for perfect match (name and order)
    are_identical = (df1_cols == df2_cols)

    # Check for names only (ignoring order and normalization)
    df1_map = {normalize_name(col): col for col in df1_cols}
    df2_map = {normalize_name(col): col for col in df2_cols}

    df1_set = set(df1_normalized)
    df2_set = set(df2_normalized)

    in_df1_only_normalized = df1_set - df2_set
    in_df2_only_normalized = df2_set - df1_set

    # Map back to original column names for the result
    in_df1_only = sorted([df1_map[name] for name in in_df1_only_normalized])
    in_df2_only = sorted([df2_map[name] for name in in_df2_only_normalized])

    # Check for order mismatch if normalized names are the same
    order_mismatch = (in_df1_only == [] and in_df2_only == []) and not are_identical

    return {
        'are_identical': are_identical,
        'in_df1_only': in_df1_only,
        'in_df2_only': in_df2_only,
        'order_mismatch': order_mismatch
    }

In [24]:
print("--- Comparing df_1 and df_2 ---")
result = compare_df_columns(df_1, df_2)
print(f"Comparison Result: {result}")
print("-" * 30)

--- Comparing df_1 and df_2 ---
Comparison Result: {'are_identical': False, 'in_df1_only': [], 'in_df2_only': [' Destination Port'], 'order_mismatch': False}
------------------------------


In [25]:
print("This is df_1 columns name:")
print(df_1.columns)

print("This is df_2 columns name:")
print(df_2.columns)

This is df_1 columns name:
Index(['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'R

In [11]:
print("This is df_1 columns name:")
print(df_1.columns)

print("This is df_2 columns name:")
print(df_2.columns)

This is df_1 columns name:
Index(['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'R

In [26]:
def _coalesce_duplicate_normalized_columns(df: pd.DataFrame, norm_func) -> pd.DataFrame:
    """
    If multiple columns normalize to the same key, coalesce them into one:
    keep the left-most column and fill its nulls from the next duplicates.
    """
    if df.empty or df.shape[1] == 0:
        return df

    # map original col -> normalized key
    norm_keys = {col: norm_func(col) for col in df.columns}

    # group columns by normalized key
    from collections import defaultdict
    groups = defaultdict(list)
    for col, key in norm_keys.items():
        groups[key].append(col)

    out = df.copy()
    cols_to_drop = []
    # for each group with >1 column, coalesce into the left-most
    for key, cols in groups.items():
        if len(cols) <= 1:
            continue
        base = cols[0]
        for c in cols[1:]:
            # fillna from the right-hand duplicate
            out[base] = out[base].where(out[base].notna(), out[c])
            cols_to_drop.append(c)
    if cols_to_drop:
        out = out.drop(columns=cols_to_drop)
    return out

def align_and_sort_dataframes(df1: pd.DataFrame, df2: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Align and sort columns of two DataFrames:
      - normalize names
      - coalesce duplicate-normalized columns within each DF
      - choose a canonical label per normalized key (prefer df1's original name)
      - rename, then reindex to the same sorted union of canonical columns
    """

    # Optional one-off removal you had
    if ' Destination Port' in df2.columns:
        df2 = df2.drop(columns=[' Destination Port'])
        print("Removed ' Destination Port' from df_2.")

    # 1) Coalesce duplicate-normalized columns within each DF
    df1 = _coalesce_duplicate_normalized_columns(df1, normalize_name)
    df2 = _coalesce_duplicate_normalized_columns(df2, normalize_name)

    # 2) Build normalized key sets
    df1_norm = {col: normalize_name(col) for col in df1.columns}
    df2_norm = {col: normalize_name(col) for col in df2.columns}

    # 3) Decide canonical labels per normalized key
    #    Preference: use df1's original column name if any; otherwise df2's
    norm_to_canon = {}

    # First pass: take df1’s original as canonical for its normalized keys
    for col, key in df1_norm.items():
        # Pick the first-seen df1 original as canonical label for this key
        norm_to_canon.setdefault(key, col)

    # Second pass: add missing keys from df2, using df2's original
    for col, key in df2_norm.items():
        norm_to_canon.setdefault(key, col)

    # 4) Build per-DF rename maps (original -> canonical)
    df1_rename = {col: norm_to_canon[df1_norm[col]] for col in df1.columns}
    df2_rename = {col: norm_to_canon[df2_norm[col]] for col in df2.columns}

    # 5) Rename
    df1_renamed = df1.rename(columns=df1_rename)
    df2_renamed = df2.rename(columns=df2_rename)

    # 6) After renaming, ensure no duplicates remain (defensive)
    if not pd.Index(df1_renamed.columns).is_unique:
        # coalesce again just in case renaming produced duplicates
        df1_renamed = _coalesce_duplicate_normalized_columns(df1_renamed, normalize_name)
        # sanity: re-run uniqueness check
        if not pd.Index(df1_renamed.columns).is_unique:
            dups = df1_renamed.columns[df1_renamed.columns.duplicated()].tolist()
            raise ValueError(f"df1 still has duplicate columns after coalescing: {dups}")

    if not pd.Index(df2_renamed.columns).is_unique:
        df2_renamed = _coalesce_duplicate_normalized_columns(df2_renamed, normalize_name)
        if not pd.Index(df2_renamed.columns).is_unique:
            dups = df2_renamed.columns[df2_renamed.columns.duplicated()].tolist()
            raise ValueError(f"df2 still has duplicate columns after coalescing: {dups}")

    # 7) Build final unified, sorted column set (canonical labels)
    all_canonical = sorted(set(df1_renamed.columns).union(df2_renamed.columns))

    # 8) Reindex both to the same columns
    df1_aligned = df1_renamed.reindex(columns=all_canonical)
    df2_aligned = df2_renamed.reindex(columns=all_canonical)

    # Ensure 'Label' column is at the end
    for df in (df1_aligned, df2_aligned):
        if "Label" in df.columns:
            lbl = df.pop("Label")
            df["Label"] = lbl

    return df1_aligned, df2_aligned

In [27]:
df_1_aligned, df_2_aligned = align_and_sort_dataframes(df_1, df_2)

print("\n--- After Alignment and Sorting ---")
print(f"df_1_aligned columns: {df_1_aligned.columns.tolist()}")
print(f"df_2_aligned columns: {df_2_aligned.columns.tolist()}")
print(f"Comparison Result: {compare_df_columns(df_1_aligned, df_2_aligned)}")
print("-" * 30)

Removed ' Destination Port' from df_2.

--- After Alignment and Sorting ---
df_1_aligned columns: ['ACK Flag Count', 'Active Max', 'Active Mean', 'Active Min', 'Active Std', 'Average Packet Size', 'Bwd Bulk Rate Avg', 'Bwd Bytes/Bulk Avg', 'Bwd Header Length', 'Bwd IAT Max', 'Bwd IAT Mean', 'Bwd IAT Min', 'Bwd IAT Std', 'Bwd IAT Total', 'Bwd Init Win Bytes', 'Bwd PSH Flags', 'Bwd Packet Length Max', 'Bwd Packet Length Mean', 'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Bwd Packet/Bulk Avg', 'Bwd Packets/s', 'Bwd Segment Size Avg', 'Bwd URG Flags', 'CWR Flag Count', 'Down/Up Ratio', 'ECE Flag Count', 'FIN Flag Count', 'FWD Init Win Bytes', 'Flow Bytes/s', 'Flow Duration', 'Flow IAT Max', 'Flow IAT Mean', 'Flow IAT Min', 'Flow IAT Std', 'Flow Packets/s', 'Fwd Act Data Pkts', 'Fwd Bulk Rate Avg', 'Fwd Bytes/Bulk Avg', 'Fwd Header Length', 'Fwd IAT Max', 'Fwd IAT Mean', 'Fwd IAT Min', 'Fwd IAT Std', 'Fwd IAT Total', 'Fwd PSH Flags', 'Fwd Packet Length Max', 'Fwd Packet Length Mean', 

In [88]:
import numpy as np

cat = np.array([0, 15, 23, 24, 26, 27, 45, 54, 59, 65, 66, 75])

np.save( "catfeaturelist.npy", cat)

In [28]:
df_1_aligned.to_csv('UNSWNB15_standardised.csv', index=False) 
df_2_aligned.to_csv('CICIDS2017_standardised.csv', index=False) 

In [2]:
import numpy as np
iat = np.array([9,10,11,12,13,31,32,33,34,40,41,42,43,44])
np.save("iatfeaturelist.npy", iat)